<a href="https://colab.research.google.com/github/duybluemind1988/Data-science/blob/master/Sonion/Sonion_process_control.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# One process in one week

In [ ]:
import datetime
import seaborn as sns; sns.set()
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
from plotly.offline import iplot
import re

from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.stats.diagnostic import acorr_ljungbox
import scipy.stats as scs
from math import sqrt
from sklearn.metrics import mean_squared_error,mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler,StandardScaler

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
path='/content/drive/My Drive/Data/Sonion/Control plan and stast/Control plan 3000/Control Plan data REC 3000 - WK34 - 2020.xlsm'

In [ ]:
import xlrd
xls = xlrd.open_workbook(path, on_demand=True)
sheet_names=xls.sheet_names()
print(xls.sheet_names())

['Master list', '39682', '38682', '30682', '37682', '30665', '30660-37660-38660', '30380-37380-38380', '37670-39670', '37662', '30650-37650 (3500T)', '37650-38655-39650', '37641', '30470-37470', '38546', '39615', '38940', '38594', '37560', '30740', '37640-38635', '30674-37674-38674 ', '30672-37672-38672', '30592', '30544-37544-38544-39544', '30538', '30530-37530-39530', '30480-37480-38481', '30370-37370-38370 ', '30750-30940-39750 _S1', '30750-30940-39750 _S2', '37750-37940_S1', '37750-37940_S2', '37751']


In [ ]:
sheet_dict={}
xls = pd.ExcelFile(path)
for name in sheet_names:
  sheet_dict[name] = pd.read_excel(xls, name)

In [ ]:
name='38682'
sheet=sheet_dict[name]

In [ ]:
#sheet.to_csv('abc.xlsx')

In [ ]:
# get information in each coil
#dim_name=sheet[sheet.columns[14]][22]
#max=sheet[sheet.columns[14]][23]
#min=sheet[sheet.columns[14]][24]
#date=pd.to_datetime(sheet[sheet.columns[9]][25:])
#dim_value=sheet[sheet.columns[14]][25:]

In [ ]:
# find begin and end col
values_col=sheet.iloc[22,:]
values_col.reset_index(drop=True,inplace=True)
begin_col=values_col[values_col=='Kích thước\nDimension'].index
begin_col=begin_col+1
begin_col

Int64Index([14], dtype='int64')

In [ ]:
end_col=values_col[values_col=='MSNV'].index
end_col

Int64Index([20], dtype='int64')

In [ ]:
sheet.columns[begin_col[0]:end_col[0]]

Index(['Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17',
       'Unnamed: 18', 'Unnamed: 19'],
      dtype='object')

In [ ]:
df_dict={}
tolerance_dict={}
for name in sheet.columns[begin_col[0]:end_col[0]]:
  df=pd.DataFrame()
  
  #tolerance_dict[sheet[name][22]]=[sheet[name][24],sheet[name][23]]
  df['Date']=sheet[sheet.columns[9]][25:]
  df['Date']=df['Date'].apply(lambda x: x.strftime("%Y %m %d %H"))
  df['Date']=pd.to_datetime(df['Date'])
  #df['Hour']=df['Date'].dt.hour
  df['Value']=sheet[name][25:]
  df['USL']=sheet[name][23] # max
  df['LSL']=sheet[name][24] # min
  df.dropna(subset=['Value'],inplace=True)
  #UCL,LCL,nominal:
  sigma=3
  df['UCL']=df['Value'].mean() + df['Value'].std()*sigma
  df['LCL']=df['Value'].mean() - df['Value'].std()*sigma
  df['Mean']=df['Value'].mean()
  df[df.columns[1:]]=df[df.columns[1:]].astype('float32')
  dim_name=sheet[name][22]
  df_dict[dim_name]=df.reset_index(drop=True)

In [ ]:
df_dict['L.Tâm Dim N']

,Date,Value,USL,LSL,UCL,LCL,Mean
0,2020-08-14 17:00:00,0.012,0.025,-0.025,0.019721,-0.004197,0.007762
1,2020-08-14 17:00:00,0.006,0.025,-0.025,0.019721,-0.004197,0.007762
2,2020-08-14 17:00:00,0.009,0.025,-0.025,0.019721,-0.004197,0.007762
3,2020-08-15 08:00:00,0.011,0.025,-0.025,0.019721,-0.004197,0.007762
4,2020-08-15 08:00:00,0.008,0.025,-0.025,0.019721,-0.004197,0.007762
5,2020-08-15 08:00:00,0.008,0.025,-0.025,0.019721,-0.004197,0.007762
6,2020-08-15 16:00:00,0.005,0.025,-0.025,0.019721,-0.004197,0.007762
7,2020-08-15 16:00:00,0.004,0.025,-0.025,0.019721,-0.004197,0.007762
8,2020-08-15 16:00:00,0.002,0.025,-0.025,0.019721,-0.004197,0.007762
9,2020-08-17 09:00:00,0.016,0.025,-0.025,0.019721,-0.004197,0.007762


In [ ]:
i=1
#Layout
fig = make_subplots(
    rows=len(df_dict), cols=1,
    #shared_xaxes=True, # share same axis
    vertical_spacing=0.05, # adjust spacing between charts
    #column_widths=[0.8, 0.2],
    subplot_titles=(list(df_dict.keys()))
)
for name in df_dict.keys(): #also group
  df=df_dict[name].copy()
  df=df.sort_values(by=['Date'])
  for a in df.columns[1:]:
    df[a] = df[a].round(decimals=3)

  df=df.groupby('Date').mean()
  #df=df.set_index('Date')
  #if start_date != '':
  #  df=df[start_date:end_date]
  #Control chart 1 
  fig.append_trace(go.Scatter(
                          x=df.index, y=df['Value'],mode='lines+markers',
                          name='mean ' + name,line=dict( color='#4280F5')
                          ),row=i, col=1
                )
  #USL, LSL
  fig.append_trace(go.Scatter(x=df.index, y=df['USL'],name='USL '+name, line=dict( color='#FF5733'),mode='lines'),row=i, col=1)
  fig.append_trace(go.Scatter(x=df.index, y=df['LSL'],name='LSL '+name,line=dict( color='#FF5733'),mode='lines'),row=i, col=1)
  #fig.append_trace(go.Scatter(x=df['Datef'], y=df['Nominal'],name='Nominal '+name,line=dict( color='#FF5733')),row=i, col=1)
  # UCL, LCL
  fig.append_trace(go.Scatter(x=df.index, y=df['UCL'],name='UCL '+name, line=dict( color='#33C2FF'),mode='lines'),row=i, col=1)
  fig.append_trace(go.Scatter(x=df.index, y=df['LCL'],name='LCL '+name, line=dict( color='#33C2FF'),mode='lines'),row=i, col=1)
  fig.append_trace(go.Scatter(x=df.index, y=df['Mean'],name='Mean '+name, line=dict( color='#33C2FF'),mode='lines'),row=i, col=1)
  i=i+1
  
fig.update_layout(height=200*len(df_dict), width=1200, title_text='Line chart')
fig.show()

In [ ]:
i=1
#Layout
fig = make_subplots(
    rows=len(df_dict), cols=1,
    #shared_xaxes=True, # share same axis
    vertical_spacing=0.05, # adjust spacing between charts
    #column_widths=[0.8, 0.2],
    subplot_titles=(list(df_dict.keys()))
)
for name in df_dict.keys(): #also group
  df=df_dict[name].copy()
  df=df.sort_values(by=['Date'])
  for a in df.columns[1:]:
    df[a] = df[a].round(decimals=3)
  df=df.set_index('Date')
  #Control chart 1 
  fig.append_trace(go.Box(
                          x=df.index, y=df['Value'],
                          name='mean ' + name, line=dict( color='#4280F5')
                          ),row=i, col=1
                )
  #USL, LSL
  fig.append_trace(go.Scatter(x=df.index, y=df['USL'],name='USL '+name, line=dict( color='#FF5733')),row=i, col=1)
  fig.append_trace(go.Scatter(x=df.index, y=df['LSL'],name='LSL '+name,line=dict( color='#FF5733')),row=i, col=1)
  #fig.append_trace(go.Scatter(x=df['Datef'], y=df['Nominal'],name='Nominal '+name,line=dict( color='#FF5733')),row=i, col=1)
  # UCL, LCL
  i=i+1
  
fig.update_layout(height=200*len(df_dict), width=1200, title_text='Line chart')
fig.show()

# All process in one week

In [ ]:
import datetime
import seaborn as sns; sns.set()
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
from plotly.offline import iplot
import re

from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.stats.diagnostic import acorr_ljungbox
import scipy.stats as scs
from math import sqrt
from sklearn.metrics import mean_squared_error,mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler,StandardScaler

In [ ]:
path='/content/drive/My Drive/Data/Sonion/Control plan and stast/Control plan 3000/Control Plan data REC 3000 - WK34 - 2020.xlsm'
import xlrd
xls = xlrd.open_workbook(path, on_demand=True)
sheet_names=xls.sheet_names()
print(xls.sheet_names())
sheet_dict={}
xls = pd.ExcelFile(path)
for name in sheet_names:
  sheet_dict[name] = pd.read_excel(xls, name)

['Master list', '39682', '38682', '30682', '37682', '30665', '30660-37660-38660', '30380-37380-38380', '37670-39670', '37662', '30650-37650 (3500T)', '37650-38655-39650', '37641', '30470-37470', '38546', '39615', '38940', '38594', '37560', '30740', '37640-38635', '30674-37674-38674 ', '30672-37672-38672', '30592', '30544-37544-38544-39544', '30538', '30530-37530-39530', '30480-37480-38481', '30370-37370-38370 ', '30750-30940-39750 _S1', '30750-30940-39750 _S2', '37750-37940_S1', '37750-37940_S2', '37751']


In [ ]:
sheet_all={}
sheet_error=[]
for name_sheet in sheet_names[1:]: # all sheet
  #print(name_sheet)
  sheet=sheet_dict[name_sheet]

  # find begin and end col
  values_col=sheet.iloc[22,:]
  values_col.reset_index(drop=True,inplace=True)
  begin_col=values_col[values_col=='Kích thước\nDimension'].index
  begin_col=begin_col+1
  end_col=values_col[values_col=='MSNV'].index
  #print(begin_col,end_col)
 
  df_dict={} # add all value, USL, LSL, UCL... in sheet
  try:
    for name in sheet.columns[begin_col[0]:end_col[0]]:
      
        df=pd.DataFrame()
        
        #tolerance_dict[sheet[name][22]]=[sheet[name][24],sheet[name][23]]
        df['Date']=sheet[sheet.columns[9]][25:]
        df['Date']=df['Date'].apply(lambda x: x.strftime("%Y %m %d %H"))
        df['Date']=pd.to_datetime(df['Date'])
        #df['Hour']=df['Date'].dt.hour
        df['Value']=sheet[name][25:]
        df['USL']=sheet[name][23] # max
        df['LSL']=sheet[name][24] # min
        df.dropna(subset=['Value'],inplace=True)
        #UCL,LCL,nominal:
        sigma=3
        df['UCL']=df['Value'].mean() + df['Value'].std()*sigma
        df['LCL']=df['Value'].mean() - df['Value'].std()*sigma
        df['Mean']=df['Value'].mean()
        df[df.columns[1:]]=df[df.columns[1:]].astype('float32')
        dim_name=sheet[name][22]
        df_dict[dim_name]=df.reset_index(drop=True)
  except:
    sheet_error.append(name_sheet)
    continue
  sheet_all[name_sheet]=df_dict
print(sheet_error)

['37560', '30740', '37751']


In [ ]:
for sheet_name in sheet_all.keys():
  df_dict=sheet_all[sheet_name]

  i=1
  #Layout
  fig = make_subplots(
      rows=len(df_dict), cols=1,
      #shared_xaxes=True, # share same axis
      #vertical_spacing=0.05, # adjust spacing between charts
      #column_widths=[0.8, 0.2],
      subplot_titles=(list(df_dict.keys()))
  )
  for name in df_dict.keys(): #also group
    df=df_dict[name].copy()
    df=df.sort_values(by=['Date'])
    for a in df.columns[1:]:
      df[a] = df[a].round(decimals=3)

    df=df.groupby('Date').mean()
    #df=df.set_index('Date')
    #if start_date != '':
    #  df=df[start_date:end_date]
    #Control chart 1 
    fig.append_trace(go.Scatter(
                            x=df.index, y=df['Value'],mode='lines+markers',
                            name='mean ' + name,line=dict( color='#4280F5')
                            ),row=i, col=1
                  )
    #USL, LSL
    fig.append_trace(go.Scatter(x=df.index, y=df['USL'],name='USL '+name, line=dict( color='#FF5733'),mode='lines'),row=i, col=1)
    fig.append_trace(go.Scatter(x=df.index, y=df['LSL'],name='LSL '+name,line=dict( color='#FF5733'),mode='lines'),row=i, col=1)
    #fig.append_trace(go.Scatter(x=df['Datef'], y=df['Nominal'],name='Nominal '+name,line=dict( color='#FF5733')),row=i, col=1)
    # UCL, LCL
    fig.append_trace(go.Scatter(x=df.index, y=df['UCL'],name='UCL '+name, line=dict( color='#33C2FF'),mode='lines'),row=i, col=1)
    fig.append_trace(go.Scatter(x=df.index, y=df['LCL'],name='LCL '+name, line=dict( color='#33C2FF'),mode='lines'),row=i, col=1)
    fig.append_trace(go.Scatter(x=df.index, y=df['Mean'],name='Mean '+name, line=dict( color='#33C2FF'),mode='lines'),row=i, col=1)
    i=i+1
    
  fig.update_layout(height=200*len(df_dict), width=1200, title_text=sheet_name)
  #fig.show()

# All process in multi weeks

In [1]:
import datetime
import seaborn as sns; sns.set()
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
from plotly.offline import iplot
import re

from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.stats.diagnostic import acorr_ljungbox
import scipy.stats as scs
from math import sqrt
from sklearn.metrics import mean_squared_error,mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler,StandardScaler

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [13]:
import glob
path='/content/drive/My Drive/Data/Sonion/Control plan and stast/Control plan 3000/'
#print(path)
all_files1=glob.glob(path + '*.xlsx')
all_files2=glob.glob(path + '*.xlsm')
all_files=all_files1+all_files2
#sort file in directory by reverse:
all_files = sorted(all_files, reverse = False)
print('number of files: ',len(all_files))

number of files:  4


In [14]:
all_files

['/content/drive/My Drive/Data/Sonion/Control plan and stast/Control plan 3000/Control Plan data REC 3000 - WK34 - 2020.xlsm',
 '/content/drive/My Drive/Data/Sonion/Control plan and stast/Control plan 3000/Control Plan data REC 3000 - WK35 - 2020.xlsm',
 '/content/drive/My Drive/Data/Sonion/Control plan and stast/Control plan 3000/Control Plan data REC 3000 - WK36 - 2020.xlsm',
 '/content/drive/My Drive/Data/Sonion/Control plan and stast/Control plan 3000/Control Plan data REC 3000 - WK37 - 2020.xlsm']

In [15]:
%%time
import xlrd
all_process_week={}
for path_name in all_files:
  week_name=path_name[-16:-5]
  xls = xlrd.open_workbook(path_name, on_demand=True)
  sheet_names=xls.sheet_names()
  #print(xls.sheet_names())
  sheet_dict={}
  xls = pd.ExcelFile(path_name)
  for name in sheet_names:
    sheet_dict[name] = pd.read_excel(xls, name)

  # Read all process in one week
  sheet_all={} # most important
  sheet_error=[]
  for name_sheet in sheet_names[1:]: # all sheet
    #print(name_sheet)
    sheet=sheet_dict[name_sheet]

    # find begin and end col
    values_col=sheet.iloc[22,:]
    values_col.reset_index(drop=True,inplace=True)
    begin_col=values_col[values_col=='Kích thước\nDimension'].index
    begin_col=begin_col+1
    end_col=values_col[values_col=='MSNV'].index
    #print(begin_col,end_col)
  
    df_dict={} # add all value, USL, LSL, UCL... in sheet
    try:
      for name in sheet.columns[begin_col[0]:end_col[0]]:
        
          df=pd.DataFrame()
          
          #tolerance_dict[sheet[name][22]]=[sheet[name][24],sheet[name][23]]
          df['Date']=sheet[sheet.columns[9]][25:]
          df['Date']=df['Date'].apply(lambda x: x.strftime("%Y %m %d %H"))
          df['Date']=pd.to_datetime(df['Date'])
          #df['Hour']=df['Date'].dt.hour
          df['Value']=sheet[name][25:]
          df['USL']=sheet[name][23] # max
          df['LSL']=sheet[name][24] # min
          df.dropna(subset=['Value'],inplace=True)
          #UCL,LCL,nominal:
          sigma=3
          df['UCL']=df['Value'].mean() + df['Value'].std()*sigma
          df['LCL']=df['Value'].mean() - df['Value'].std()*sigma
          df['Mean']=df['Value'].mean()
          df[df.columns[1:]]=df[df.columns[1:]].astype('float32')
          dim_name=sheet[name][22]
          df_dict[dim_name]=df.reset_index(drop=True)
    except:
      sheet_error.append(name_sheet)
      continue
    sheet_all[name_sheet]=df_dict
  all_process_week[week_name]=sheet_all
  print(week_name,sheet_error)

WK34 - 2020 ['37560', '30740', '37751']
WK35 - 2020 ['30740', '37750-37940_S2', '37751']
WK36 - 2020 ['37560', '30740', '37751', '37312-38312']
WK37 - 2020 ['37560', '30740', '37751', '37312-38312']
CPU times: user 14.4 s, sys: 29 ms, total: 14.5 s
Wall time: 14.5 s


In [16]:
all_process_week.keys()

dict_keys(['WK34 - 2020', 'WK35 - 2020', 'WK36 - 2020', 'WK37 - 2020'])

In [17]:
all_process_week['WK34 - 2020'].keys()

dict_keys(['39682', '38682', '30682', '37682', '30665', '30660-37660-38660', '30380-37380-38380', '37670-39670', '37662', '30650-37650 (3500T)', '37650-38655-39650', '37641', '30470-37470', '38546', '39615', '38940', '38594', '37640-38635', '30674-37674-38674 ', '30672-37672-38672', '30592', '30544-37544-38544-39544', '30538', '30530-37530-39530', '30480-37480-38481', '30370-37370-38370 ', '30750-30940-39750 _S1', '30750-30940-39750 _S2', '37750-37940_S1', '37750-37940_S2'])

In [19]:
all_process_week['WK35 - 2020']['39682'].keys()

dict_keys(['Chiều rộng Dim E', 'Chiều dài Dim C', 'Cung hint Dim I', 'Cung J Dim J', 'Offset Dim M'])

In [20]:
base_week=all_process_week[list(all_process_week.keys())[0]]
base_week.keys()

dict_keys(['39682', '38682', '30682', '37682', '30665', '30660-37660-38660', '30380-37380-38380', '37670-39670', '37662', '30650-37650 (3500T)', '37650-38655-39650', '37641', '30470-37470', '38546', '39615', '38940', '38594', '37640-38635', '30674-37674-38674 ', '30672-37672-38672', '30592', '30544-37544-38544-39544', '30538', '30530-37530-39530', '30480-37480-38481', '30370-37370-38370 ', '30750-30940-39750 _S1', '30750-30940-39750 _S2', '37750-37940_S1', '37750-37940_S2'])

In [21]:
# concat all process in each weeks based on keys value:
%%time
for week_name in list(all_process_week.keys())[1:]: # ko tính base week nên từ 1: 
  other_week=all_process_week[week_name]
  for process_name in other_week.keys(): # dict all process
    for process_name_base in base_week.keys(): # dict all process
        if process_name_base==process_name:
          for dim_name in base_week[process_name_base].keys(): # dict all dim in each process
             base_week[process_name_base][dim_name]=pd.concat([base_week[process_name_base][dim_name],other_week[process_name_base][dim_name]])
    #print(process_name)


CPU times: user 337 ms, sys: 7.2 ms, total: 344 ms
Wall time: 336 ms


In [22]:
base_week

{'30370-37370-38370 ': {'3000 Dim A':                    Date  Value    USL   LSL       UCL       LCL      Mean
  0   2020-08-14 20:00:00  0.077  0.095  0.06  0.089987  0.074066  0.082027
  1   2020-08-14 20:00:00  0.079  0.095  0.06  0.089987  0.074066  0.082027
  2   2020-08-14 20:00:00  0.085  0.095  0.06  0.089987  0.074066  0.082027
  3   2020-08-14 20:00:00  0.086  0.095  0.06  0.089987  0.074066  0.082027
  4   2020-08-14 20:00:00  0.089  0.095  0.06  0.089987  0.074066  0.082027
  ..                  ...    ...    ...   ...       ...       ...       ...
  135 2020-09-08 21:00:00  0.079  0.095  0.06  0.088742  0.073015  0.080879
  136 2020-09-08 21:00:00  0.084  0.095  0.06  0.088742  0.073015  0.080879
  137 2020-09-08 21:00:00  0.078  0.095  0.06  0.088742  0.073015  0.080879
  138 2020-09-08 21:00:00  0.083  0.095  0.06  0.088742  0.073015  0.080879
  139 2020-09-08 21:00:00  0.080  0.095  0.06  0.088742  0.073015  0.080879
  
  [948 rows x 7 columns],
  '3700 Dim A':        